# Using tool to get information about a country, from wikipedia




## Import libaries and prepare the API credential

In [13]:
import tempfile
import shutil
import os
import subprocess
import requests

from autogen import ConversableAgent, register_function
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## A Python function to call Wiki API


In [14]:
def describe_country(country_name:str)->str:
    url = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles="+country_name
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        json_data = response.json()  # Extract JSON content
        page = next(iter(json_data["query"]["pages"].values()))
        return page["extract"]
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")


## Agent 1: tool executor (just to wrap up the function)

In [15]:

wiki_tool = ConversableAgent(
    "wiki_tool",
    system_message = "You are a helpful assistant. You may decide to use some external tools if necessary",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False
)


### Agent 2: Tool driver (caller of the function)

In [16]:
wiki_tool_driver = ConversableAgent(
    "wiki_tool_driver",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False
)

### Register the tool to both caller and executor

*Question:* Why we need tool agents to use a tool?

In [17]:
register_function(
    describe_country,
    caller = wiki_tool_driver,
    executor = wiki_tool,
    description = "get the description of a country from wiki. this service only receive the formal name of a country"
)

## Now we start

In [18]:
wiki_tool.initiate_chat(
    wiki_tool_driver,
    message = "I want to know something about the united states",
    max_turns = 2
)


wiki_tool (to wiki_tool_driver):

I want to know something about the united states

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
wiki_tool_driver (to wiki_tool):

***** Suggested tool call (call_9IeUf0dCYcHZF9DDPgCpGDk2): describe_country *****
Arguments: 
{"country_name":"United States"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION describe_country...
wiki_tool (to wiki_tool_driver):

wiki_tool (to wiki_tool_driver):

***** Response from calling tool (call_9IeUf0dCYcHZF9DDPgCpGDk2) *****
The United States of America (USA or U.S.A.), commonly known as the United States (US or U.S.) or America, is a country primarily located in North America. It is a federation of 50 states, which also includes its federal capital district of Washington, D.C., and

ChatResult(chat_id=None, chat_history=[{'content': 'I want to know something about the united states', 'role': 'assistant'}, {'tool_calls': [{'id': 'call_9IeUf0dCYcHZF9DDPgCpGDk2', 'function': {'arguments': '{"country_name":"United States"}', 'name': 'describe_country'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': "The United States of America (USA or U.S.A.), commonly known as the United States (US or U.S.) or America, is a country primarily located in North America. It is a federation of 50 states, which also includes its federal capital district of Washington, D.C., and 326 Indian reservations. The 48 contiguous states border Canada to the north and Mexico to the south. The State of Alaska is non-contiguous and lies to the northwest, while the State of Hawaii is an archipelago in the Pacific Ocean. Outside the union of states, the U.S. also asserts sovereignty over five major unincorporated island territories and various uninhabited islands. The country 